In [ ]:
from functions import *
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks", color_codes=True)
import pickle
import re
from sklearn.metrics.pairwise import cosine_similarity



In [1]:
from functions import *
logger = get_logger('log_experiments')
seed_everything(seed=42)

def run_experiment_gnn(database_name, 
                       K, 
                       hidden_channels, 
                       num_layers, 
                       p_dropout, 
                       Kc,
                       docf, 
                       docst,
                       activation, 
                       version,
                       num_epochs):
    
    try:
        
        heterodata_description = f'{database_name}_K_{K}_Kc_{Kc}_docf_{str(docf)}_docst_{str(docst)}'
        with open(f'./pickle_objects/preprocess/{database_name}/heterodata_pbg_{heterodata_description}_train.pickle', 'rb') as f:
            heterodata_train = pickle.load(f)
        with open(f'./pickle_objects/preprocess/{database_name}/heterodata_pbg_{heterodata_description}_test.pickle', 'rb') as f:
            heterodata_test = pickle.load(f)
                        
        logger.info(f'Loaded preprocessed heterographs {heterodata_description}.')

        training_description = f'{heterodata_description}_act_{activation}_ver_{version}'

        df_experiment = run_heterognn_splitted(database_name=database_name,
                           description=training_description, 
                           heterodata_train=heterodata_train,
                           heterodata_test=heterodata_test,
                           hidden_channels=hidden_channels,
                           num_layers=num_layers,
                           p_dropout=p_dropout,
                           num_epochs=num_epochs, 
                           aggr='sum',
                           version=version, 
                           activation=activation, 
                           verbose=True)
        
        loss_test, micro_test, acc_test, epoch_convergence = df_experiment
        df = pd.DataFrame(columns=['database_name', 'K', 'Kc', 'docf', 'docst', 'hidden_channels', 'num_layers', 'p_dropout', 'activation', 'version', 'loss_test', 'micro_test', 'acc_test', 'epoch_convergence'])
        output_list = [database_name, K, Kc, docf, docst, hidden_channels, num_layers, p_dropout, activation, version, loss_test, micro_test, acc_test, epoch_convergence]
        row = pd.Series(output_list, index=df.columns)
        df = df.append(row,ignore_index=True) 
        
#         with open('./csv_objects/summary/experiments_summary.csv', 'a') as f:
#             df.to_csv(f, mode='a', sep=';', decimal=',', index=False, header=f.tell()==0)
        
        logger.info(f'Executed experiments on {database_name} {training_description}. Results saved as pickle objects.')

    except Exception as e:
        logger.info(f'Error occurred: \n{e}')
        pass

if __name__ == '__main__':

    databse_list = ['classic4']#['20ng', '20ng', 'bbcnews', 'reuters', 'classic4', 'nsf', 'webkb', 'agnews']
    K = 50
    hidden_channels = 400
    num_layers = 3
    p_dropout = 0.2
    Kc = 400
    docf = 'replace'
    docst = 0.5
    num_epochs = 1500
    activation_list = ['fl']#['ce', 'fl']
    gnn_version_list = [1, 2, 3, 4]
    

    logger.info('Running experiments on datasets with heterographs and GNNs.')

    for database_name in databse_list:
        for activation in activation_list:
            for version in gnn_version_list:
                run_experiment_gnn(database_name, 
                                  K, 
                                  hidden_channels, 
                                  num_layers, 
                                  p_dropout, 
                                  Kc,
                                  docf, 
                                  docst,
                                  activation, 
                                  version,
                                  num_epochs)

!zip -r /content/csv_objects.zip /content/csv_objects
!cp /content/csv_objects.zip  /content/drive/MyDrive/project 


[INFO][2023-04-30 14:12:21 - Mod: 125686976 - Func: <module> - Line: 72]: Running experiments on datasets with heterographs and GNNs.
[INFO][2023-04-30 14:12:21 - Mod: 125686976 - Func: run_experiment_gnn - Line: 25]: Loaded preprocessed heterographs classic4_K_50_Kc_400_docf_replace_docst_0.5.
  2%|▊                                      | 31/1500 [01:32<1:13:10,  2.99s/it][INFO][2023-04-30 14:13:57 - Mod: functions - Func: run_heterognn_splitted - Line: 696]: Loss (train): 0.4233, Loss (test): 2.0810, F1 (train): 0.6783, F1 (test): 0.1563


  3%|█▏                                     | 48/1500 [02:22<1:12:46,  3.01s/it][ERROR][2023-04-30 14:14:47 - Mod: gnn - Func: test_model - Line: 131]: Error testing model: 
 The predictions values should be between 0 and 1,                 make sure to pass the values to sigmoid for binary                 classification or softmax for multi-class classification
  3%|█▏                                     | 48/1500 [02:25<1:13:29,  3.04s/it]
[ERROR][2023-04-30 14:14:47 - Mod: functions - Func: run_heterognn_splitted - Line: 711]: Error training model on heterodata: 
 The predictions values should be between 0 and 1,                 make sure to pass the values to sigmoid for binary                 classification or softmax for multi-class classification
[INFO][2023-04-30 14:14:47 - Mod: 125686976 - Func: run_experiment_gnn - Line: 54]: Error occurred: 
cannot unpack non-iterable NoneType object
[INFO][2023-04-30 14:14:47 - Mod: 125686976 - Func: run_experiment_gnn - Line: 25]: Loaded p

  2%|▌                                      | 23/1500 [02:46<2:58:39,  7.26s/it]


KeyboardInterrupt: 